In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
plt.rcParams['font.size'] = 10
plt.rcParams['lines.linewidth'] = 3
%matplotlib inline

# Advanced Pandas
Padas will be our main tool for data management and data analysis. Here we explore a few advanced topics on how to use Pandas efficiently:
1. How to **group** observations and perform calculations on the different sub-groups of data
1. How to **combine** different datasets
1. How to use logical operators to **select** groups of observations

___
## Part 1 - GroupBy Operations
Pandas has powerful and flexible **grouping capabilities**. They work similarly to pivot tables in Microsoft Excel where we can group observations based on some varaible and then we can perform calculations on the different groups. Let's start by loading our data on size and beta of US listed companies

In [12]:
shares = pd.read_csv('shares.csv', index_col='ticker')
shares.head()

,name,industry,size,ret,beta
ticker,,,,,
AAPL,APPLE INC,Business Equipment,643.12012,0.236708,1.289383
XOM,EXXON MOBIL CORP,"Oil, Gas, and Coal",391.48221,-0.016955,0.493250
MSFT,MICROSOFT CORP,Business Equipment,381.72610,0.304496,1.195665
JNJ,JOHNSON & JOHNSON,Healthcare,292.70294,0.098696,0.461693
WFC,WELLS FARGO & CO NEW,Finance,284.38553,0.027379,0.839797


The following code groups by the `industry` column and outputs statistics for the single groups:

In [13]:
industries = shares.groupby('industry')

The result of this operation is a weird object that cannot be visualized. It is only useful to perform operations on the groups

In [14]:
industries

We can start by counting how many observations (companies) there are in every group using the [**`GroupBy.size()`**](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.GroupBy.size.html). Please notice that "size" here is a method of the groupby operator and has nothing to do with the column called `size` in the original DataFrame.

In [15]:
industries.size() 

industry
Business Equipment       197
Chemicals                 57
Consumer Durables         33
Consumer non Durables     98
Finance                  246
Healthcare               128
Manufacturing            197
Oil, Gas, and Coal        72
Other                    177
Retail                   141
Telecommunications        40
Utilities                 57
dtype: int64

Now that our companies are grouped by industry we can easily measure the **average beta by industry**. We just need to apply the `.mean()` method to the groupby object. 

In [16]:
industries['beta'].mean()

industry
Business Equipment       1.626038
Chemicals                1.011927
Consumer Durables        1.402230
Consumer non Durables    0.921691
Finance                  0.889086
Healthcare               1.056634
Manufacturing            1.192749
Oil, Gas, and Coal       1.043445
Other                    1.115744
Retail                   1.117096
Telecommunications       1.324042
Utilities                0.411085
Name: beta, dtype: float64

We can use the generic [**`GroupBy.agggregate()`**](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.DataFrameGroupBy.aggregate.html) method to ask for any number of arbitrary group statistics. For example here we want to learn the best, worst and average performance of the firms in each industry:

In [17]:
industries['ret'].aggregate([min, max, np.mean])

,min,max,mean
industry,,,
Business Equipment,-0.310205,0.765975,0.108934
Chemicals,-0.614735,0.336840,0.037285
Consumer Durables,-0.258810,0.225435,0.034719
Consumer non Durables,-0.668633,0.293459,0.043510
Finance,-0.673303,0.325035,0.084814
Healthcare,-0.617889,0.578339,0.019449
Manufacturing,-0.447244,0.431358,0.060195
"Oil, Gas, and Coal",-0.617344,0.312777,-0.136099
Other,-0.584756,0.459846,0.052756


___
#### Small Coding Issue
In the code here above, we have used the `DataFrame.Groupby()` operator to create an intermediate object, `industries`, and then we have used this object to perform all the others calculations. Strictly speaking **this is not necessary**, we can avoid the creation of this intermediate object by _chaining_ our calculation methods, such as `.mean()`, directly to the `DataFrame.Groupby()` operator as follows

In [18]:
shares.groupby('industry').size()

industry
Business Equipment       197
Chemicals                 57
Consumer Durables         33
Consumer non Durables     98
Finance                  246
Healthcare               128
Manufacturing            197
Oil, Gas, and Coal        72
Other                    177
Retail                   141
Telecommunications        40
Utilities                 57
dtype: int64

In [9]:
shares.groupby('industry')['beta'].mean()

industry
Business Equipment       1.626038
Chemicals                1.011927
Consumer Durables        1.402230
Consumer non Durables    0.921691
Finance                  0.889086
Healthcare               1.056634
Manufacturing            1.192749
Oil, Gas, and Coal       1.043445
Other                    1.115744
Retail                   1.117096
Telecommunications       1.324042
Utilities                0.411085
Name: beta, dtype: float64

___
### Exercise 5.01
The file `tesla_month.csv` contains monthly return for Tesla (from 2010 to the end of 2020). The file also indicates the calendar month of each observation
* Load the data in a DataFrame called `tesla_data`indexed by the column `date`
* Show on screen the first lines of `tesla_data`

In [4]:
tesla_data = pd.read_csv('tesla_month.csv', index_col='date')
tesla_data.head()

,month,ret
date,,
30jul2010,7,-0.163240
31aug2010,8,-0.023069
30sep2010,9,0.047485
29oct2010,10,0.070326
30nov2010,11,0.617674


**Group the DataFrame by calendar month** and create an object called `mean_ret` with the **annualized mean return** for every calendar month (annualize the returns by simply multiplying them by 12).

In [15]:
mean_ret = tesla_data.groupby('month')['ret'].mean()
mean_ret 
mean_ret * 12



month
1     0.891550
2     0.540444
3    -0.132749
4     1.355619
5     0.873019
6     1.279513
7     0.141083
8     1.077492
9    -0.187213
10    0.347365
11    1.411360
12    0.390980
Name: ret, dtype: float64

Create an object called `std_ret` with the **annualized standard deviation** for every calendar month (annualize the standard deviation by simply multiplying them by the squred root of 12).

In [33]:
std_ret = tesla_data.groupby('month')['ret'].std()* np.sqrt(12)
std_ret



month
1     0.750011
2     0.420921
3     0.562179
4     0.728184
5     0.969757
6     0.376590
7     0.559631
8     0.859891
9     0.296676
10    0.576737
11    0.821465
12    0.556751
Name: ret, dtype: float64

Using the two objects `mean_ret` and `std_ret`, calculate the Sharpe Ratio of Tesla in every calendar month assuming a risk free rate of 1%. For simplicity let's use the simple definition of Sharpe ratio:

$$ S = \frac{E[r] - rf}{\sigma_{r}} $$

In [20]:
sharpe = (mean_ret - 0.01)/std_ret 
sharpe


month
1     0.296966
2     0.288348
3    -0.129785
4     0.489838
5     0.224157
6     0.888824
7     0.010875
8     0.321441
9    -0.298928
10    0.113803
11    0.453803
12    0.140503
Name: ret, dtype: float64

___
## Part 2 - Concatenation, Joining, and Merging
This section walks through different approaches to combine two simple data sets in the form of DataFrame objects. The two simple data sets are:

In [10]:
shares_1 = pd.read_csv('shares_1.csv', index_col='ticker')
shares_1

,size
ticker,
AAPL,643.1201
XOM,391.4822
MSFT,381.7261
JNJ,292.7029
WFC,284.3855


In [11]:
shares_2 = pd.read_csv('shares_2.csv', index_col='ticker')
shares_2

,beta
ticker,
AAPL,1.28938
XOM,0.49325
MSFT,1.19567
WMT,0.52437
GE,1.18979


#### Concatenation
Concatenation or appending basically means that **rows are added** from one DataFrame object to another one. This can be accomplished via the The **[`pandas.concat()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html)** function.

In [12]:
shares_concat = pd.concat((shares_1, shares_2), sort=False)
shares_concat

,size,beta
ticker,,
AAPL,643.1201,NaN
XOM,391.4822,NaN
MSFT,381.7261,NaN
JNJ,292.7029,NaN
WFC,284.3855,NaN
AAPL,NaN,1.28938
XOM,NaN,0.49325
MSFT,NaN,1.19567
WMT,NaN,0.52437


This concatenation does not do exactly what we want. The two frames are just put one on top of the other. Sometimes this is what we are looking for, but here we end up, for example, with two different rows for `AAPL` and `MSFT` each one with only one variable present.

#### Joining
When joining the two data sets using the [**`DataFrame.join()`**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html) method, the sequence of the DataFrame objects also matters but in a different way. **Only the index values from the first DataFrame** object are used. This default behavior is called a **left join**:

In [13]:
shares_joined = shares_1.join(shares_2)
shares_joined

,size,beta
ticker,,
AAPL,643.1201,1.28938
XOM,391.4822,0.49325
MSFT,381.7261,1.19567
JNJ,292.7029,NaN
WFC,284.3855,NaN


As you can see `shares_1` **is on the left** and thus has control of the joining. Only the index items from this DataFrame are used. If we change the order we will get a different result.

In [14]:
shares_joined = shares_2.join(shares_1)
shares_joined

,beta,size
ticker,,
AAPL,1.28938,643.1201
XOM,0.49325,391.4822
MSFT,1.19567,381.7261
WMT,0.52437,NaN
GE,1.18979,NaN


There are a total of four different join methods available, each leading to a different behavior with regard to how index values and the corresponding data rows are handled. The default one is the **left join** where the DataFrame "on the left" decides which index items are retained:

In [15]:
shares_joined = shares_1.join(shares_2, how='left')
shares_joined

,size,beta
ticker,,
AAPL,643.1201,1.28938
XOM,391.4822,0.49325
MSFT,381.7261,1.19567
JNJ,292.7029,NaN
WFC,284.3855,NaN


With a **right join** we can give control to the DataFrame on the right without having to change the order:

In [16]:
shares_joined = shares_1.join(shares_2, how='right')
shares_joined

,size,beta
ticker,,
AAPL,643.1201,1.28938
XOM,391.4822,0.49325
MSFT,381.7261,1.19567
WMT,NaN,0.52437
GE,NaN,1.18979


With an **inner join** only the intersection of the two frames is retained, the items that appear in both:

In [17]:
shares_joined = shares_1.join(shares_2, how='inner')
shares_joined

,size,beta
ticker,,
AAPL,643.1201,1.28938
XOM,391.4822,0.49325
MSFT,381.7261,1.19567


Finally, with an **outer join** all the items from both DataFrames are retained and combined:

In [18]:
shares_joined = shares_1.join(shares_2, how='outer')
shares_joined

,size,beta
ticker,,
AAPL,643.1201,1.28938
GE,NaN,1.18979
JNJ,292.7029,NaN
MSFT,381.7261,1.19567
WFC,284.3855,NaN
WMT,NaN,0.52437
XOM,391.4822,0.49325


#### Merging
While a **join** operation takes place based **on the indices** of the DataFrame objects to be joined, a **merge** operation typically takes place **on a column shared** between the two data sets using the funcion [**`pandas.merge()`**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.merge.html). To this end, let's load the data again, this time we keep the ticker as an additional column:

In [19]:
shares_1b = pd.read_csv('shares_1.csv')
shares_1b

,ticker,size
0,AAPL,643.1201
1,XOM,391.4822
2,MSFT,381.7261
3,JNJ,292.7029
4,WFC,284.3855


In [20]:
shares_2b = pd.read_csv('shares_2.csv')
shares_2b

,ticker,beta
0,AAPL,1.28938
1,XOM,0.49325
2,MSFT,1.19567
3,WMT,0.52437
4,GE,1.18979


By default, the merge operation takes place based **on all the columns that share the same name** in the two DataFrames.

In [21]:
shares_merged = pd.merge(shares_1b,shares_2b)
shares_merged

,ticker,size,beta
0,AAPL,643.1201,1.28938
1,XOM,391.4822,0.49325
2,MSFT,381.7261,1.19567


If we have multiple common columns but want to merge on a specific one, **we can specify** the merging column(s)

In [22]:
shares_merged = pd.merge(shares_1b,shares_2b, on='ticker')
shares_merged

,ticker,size,beta
0,AAPL,643.1201,1.28938
1,XOM,391.4822,0.49325
2,MSFT,381.7261,1.19567


The merging column(s) **do not have to have the same name** in the two DataFrames

In [23]:
shares_merged = pd.merge(shares_1b,shares_2b, left_on='ticker', right_on='ticker')
shares_merged

,ticker,size,beta
0,AAPL,643.1201,1.28938
1,XOM,391.4822,0.49325
2,MSFT,381.7261,1.19567


We can finally also specify whether the merger should be **left, right, outer or inner,** with the latter being the default.

In [24]:
shares_merged = pd.merge(shares_1b,shares_2b, on='ticker', how='outer')
shares_merged

,ticker,size,beta
0,AAPL,643.1201,1.28938
1,XOM,391.4822,0.49325
2,MSFT,381.7261,1.19567
3,JNJ,292.7029,NaN
4,WFC,284.3855,NaN
5,WMT,NaN,0.52437
6,GE,NaN,1.18979


___
### Exercise 5.02
The file `tesla_ret.csv` contains monthly returns for Tesla, while `tesla_vol.csv` contains monthly trading volumes. Load these files into two DataFrames (`tesla_ret` and `tesla_vol`). Both DataFrames should be indexed by the column `time`.

In [42]:
tesla_1 = pd.read_csv('tesla_ret.csv', index_col='time')
tesla_1

tesla_2 = pd.read_csv('tesla_vol.csv', index_col='time')
tesla_2

tesla_concat = pd.concat((tesla_1, tesla_2), sort=False)
tesla_concat



,month,ret,volume
time,,,
30jul2010,7,-0.163240,NaN
31aug2010,8,-0.023069,NaN
30sep2010,9,0.047485,NaN
29oct2010,10,0.070326,NaN
30nov2010,11,0.617674,NaN
...,...,...,...
31aug2020,8,NaN,4051970.0
30sep2020,9,NaN,17331954.0
30oct2020,10,NaN,8330610.0


Create a DataFrame `tesla` by **joining** `tesla_ret` and `tesla_vol`. 

**WARNING:** while [**`DataFrame.join()`**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html) is perfectly appropriate here (we want to match observations on the date variable contained in the index) there is one additional complication: the two DataFrames have a column with the same name... This will create an error in the simple joining function because **the resulting DataFrame would have two columns with the same name** and this is not allowed! To solve this issue look at the help page for [**`DataFrame.join()`**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html) at the `lsuffix` and `rsuffix` parameters.

In [49]:
tesla_joined = tesla_1.join(tesla_2, lsuffix = 'left' )
tesla_joined


,monthleft,ret,month,volume
time,,,,
30jul2010,7,-0.163240,7,657886
31aug2010,8,-0.023069,8,153193
30sep2010,9,0.047485,9,183742
29oct2010,10,0.070326,10,67145
30nov2010,11,0.617674,11,288315
...,...,...,...,...
31aug2020,8,0.741452,8,4051970
30sep2020,9,-0.139087,9,17331954
30oct2020,10,-0.095499,10,8330610


If you have done your job properly, you should have two columns with **slightly different name but the same content** (the calendar month). Now drop one of the two columns (look at the [**`DataFrame.drop()`**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html) method.

In [52]:
tesla = tesla_joined.drop(columns = 'monthleft')
tesla

,ret,month,volume
time,,,
30jul2010,-0.163240,7,657886
31aug2010,-0.023069,8,153193
30sep2010,0.047485,9,183742
29oct2010,0.070326,10,67145
30nov2010,0.617674,11,288315
...,...,...,...
31aug2020,0.741452,8,4051970
30sep2020,-0.139087,9,17331954
30oct2020,-0.095499,10,8330610


Finally change the name of the remaining column back to `month` using the [**`DataFrame.rename()`**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rename.html) method.

In [58]:
monthleft = {**'DataFrame.rename'()**}


SyntaxError: invalid syntax (3393626092.py, line 1)

___
## Part 3 - Complex Selection
Often, data selection is accomplished by formulation of conditions on column values, and potentially combining multiple such conditions logically. Let's reload again our intitial DataFrame:

In [25]:
shares = pd.read_csv('shares.csv', index_col='ticker')
shares.head()

,name,industry,size,ret,beta
ticker,,,,,
AAPL,APPLE INC,Business Equipment,643.12012,0.236708,1.289383
XOM,EXXON MOBIL CORP,"Oil, Gas, and Coal",391.48221,-0.016955,0.493250
MSFT,MICROSOFT CORP,Business Equipment,381.72610,0.304496,1.195665
JNJ,JOHNSON & JOHNSON,Healthcare,292.70294,0.098696,0.461693
WFC,WELLS FARGO & CO NEW,Finance,284.38553,0.027379,0.839797


Let's see how a logical condition works in Pandas. Let's ask **whether betas are greater than one**:

In [26]:
shares['beta']>1

ticker
AAPL      True
XOM      False
MSFT      True
JNJ      False
WFC      False
         ...  
BIOS      True
PVA      False
NLS       True
PHIIK    False
CECO     False
Name: beta, Length: 1443, dtype: bool

This returns a series of boolean operators (`True` or `False`) that tell us, for each company, whether the beta is greater than one. We can use this series to **select only the companies with high beta**.

In [27]:
shares[shares['beta']>1]

,name,industry,size,ret,beta
ticker,,,,,
AAPL,APPLE INC,Business Equipment,643.120120,0.236708,1.289383
MSFT,MICROSOFT CORP,Business Equipment,381.726100,0.304496,1.195665
GE,GENERAL ELECTRIC CO,Manufacturing,253.766190,-0.121686,1.189791
JPM,JPMORGAN CHASE & CO,Finance,233.935870,0.204319,1.381404
ORCL,ORACLE CORP,Business Equipment,197.479780,0.049413,1.428508
...,...,...,...,...,...
XOXO,X O GROUP INC,Consumer non Durables,0.489539,0.136323,1.405959
FORM,FORMFACTOR INC,Business Equipment,0.486055,0.247368,1.265079
EXAR,EXAR CORP,Business Equipment,0.480165,0.049870,1.230422


This command returns the portion of the original DataFrame with only the high-risk companies. Of course, we can **combine multiple logical conditions**. For example, let's ask for the high-risk company in the _finance_ industry. We need to use the [`&` logical operator](https://pandas.pydata.org/docs/user_guide/indexing.html#boolean-indexing) to combine the conditions and each condition should be in parenthesis (don't ask me why...).

In [28]:
shares[(shares['beta']>1) & (shares['industry']=='Finance')]

,name,industry,size,ret,beta
ticker,,,,,
JPM,JPMORGAN CHASE & CO,Finance,233.935870,0.204319,1.381404
BAC,BANK OF AMERICA CORP,Finance,188.139280,0.164614,1.351432
AXP,AMERICAN EXPRESS CO,Finance,96.266350,0.076522,1.366297
GS,GOLDMAN SACHS GROUP INC,Finance,84.421883,0.050442,1.532228
AIG,AMERICAN INTERNATIONAL GROUP INC,Finance,78.409073,0.004156,1.790988
...,...,...,...,...,...
HAFC,HANMI FINANCIAL CORP,Finance,0.695957,0.013905,1.175250
RECN,RESOURCES CONNECTION INC,Finance,0.619260,0.027879,1.232413
AI,ARLINGTON ASSET INVESTMENT CORP,Finance,0.599204,-0.132750,1.826897


We can of course **concatenate as many conditions** as we want. Here we ask for high-risk finance companies with market capitalization greater than $100b:

In [29]:
shares[(shares['beta']>1) & (shares['industry']=='Finance') & (shares['size']>100)]

,name,industry,size,ret,beta
ticker,,,,,
JPM,JPMORGAN CHASE & CO,Finance,233.93587,0.204319,1.381404
BAC,BANK OF AMERICA CORP,Finance,188.13928,0.164614,1.351432


___
### Exercise 5.03
Using the DataFrame `shares` introduced in this notebook calculate the average size of all the companies in the `Healthcare` industry

If you have done your calculation correctly, the answer should be around `16.8616`. Is the average size affected by beta? Calculate the average size of healthcare companies with beta above (or equal to) one and with beta below one.

If you have done your calculations correctly, you should see that high risk companies have a much smaller average size (`7.737`) compared to low risk companies (`25.986`). 

To have a more robust understanding of the relationship between size and risk **calculate the correlation** between the column `size` and the column `beta` in the DataFrame `df` considering only healthcare companies. 

**HINT:** The method [**`DataFrame.corr()`**](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html) automatically calculates the correlation between all the (numerical) columns in a DataFrame.

If you have done the calculation correctly you should have a confirmation of the negative relationship between size and risk, with a correlation coefficient of `-0.260`.

___
## Additional Exercises

### Exercise 5.04
Using the data in the DataFrame `shares` created in this notebook, generate a series called `finance` with the beta of all the companies in the finance industry.

Now that you have the series, plot an histogram to show the distribution of the betas. The graph should be titled _"Financial Companies"_

Now transform `finance` into a DataFrame. To achieve this, you may want to have a look at the method **[`Series.to_frame()`](https://pandas.pydata.org/docs/reference/api/pandas.Series.to_frame.html)**. 

**ATTENTION** This point will ask you to learn new python functions using the help files.

Once you have a DataFrame, add a column called `Description` with the word `High` if the beta is higher than `1` and `Low` if the beta is lower than `1`. An easy way to change the value of a column based on a logical condition is the NumPy function **[`np.where()`](https://numpy.org/doc/stable/reference/generated/numpy.where.html)**. Alternatively you can have a look at the method **[`Series.mask()`](https://pandas.pydata.org/docs/reference/api/pandas.Series.mask.html)**.